In [5]:
import streamlit as st
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm



# Load the data
data_path = r'สถิติ Pose Repairman.xlsx'
df = pd.read_excel(data_path, sheet_name="ข้อมูลการใช้นำยา", engine='openpyxl')


In [6]:
df

,แผนก,หมายเลขเครื่อง,ปัญหา,ระยะเวลาในใช้น้ำยา /แบต (วัน),วันที่
0,WARD 5C,2,น้ำยาหมด,23,2024-06-30
1,WARD 5C,3,น้ำยาหมด,28,2024-06-30
2,WARD 5C,3,น้ำยาหมด,30,2024-06-30
3,WARD 5C,2,น้ำยาหมด,40,2024-06-30
4,WARD 5C,3,น้ำยาหมด,25,2024-06-30
...,...,...,...,...,...
2297,KIDNEY(ศูนย์โรคไต),1473,แบตเตอร์รี่หมด,153,2024-01-08
2298,KIDNEY(ศูนย์โรคไต),1474,แบตเตอร์รี่หมด,169,2024-06-18
2299,KIDNEY(ศูนย์โรคไต),1475,แบตเตอร์รี่หมด,156,2024-05-30
2300,KIDNEY(ศูนย์โรคไต),1476,แบตเตอร์รี่หมด,157,2024-05-30


In [22]:
df
df = df.dropna()


In [43]:
df

,แผนก,หมายเลขเครื่อง,ปัญหา,ระยะเวลาในใช้น้ำยา /แบต (วัน),วันที่
0,WARD 5C,2,น้ำยาหมด,23,2024-06-30
1,WARD 5C,3,น้ำยาหมด,28,2024-06-30
2,WARD 5C,3,น้ำยาหมด,30,2024-06-30
3,WARD 5C,2,น้ำยาหมด,40,2024-06-30
4,WARD 5C,3,น้ำยาหมด,25,2024-06-30
...,...,...,...,...,...
2297,KIDNEY(ศูนย์โรคไต),1473,แบตเตอร์รี่หมด,153,2024-01-08
2298,KIDNEY(ศูนย์โรคไต),1474,แบตเตอร์รี่หมด,169,2024-06-18
2299,KIDNEY(ศูนย์โรคไต),1475,แบตเตอร์รี่หมด,156,2024-05-30
2300,KIDNEY(ศูนย์โรคไต),1476,แบตเตอร์รี่หมด,157,2024-05-30


In [24]:
# Display DataFrame columns for debugging
st.write("DataFrame Columns:")
st.write(df.columns)

In [25]:
# Define column names
department_column = 'แผนก'
machine_id_column = 'หมายเลขเครื่อง'
issue_column = 'ปัญหา'
maintenance_duration_column = 'ระยะเวลาในใช้น้ำยา /แบต (วัน)'


In [26]:
# Ensure correct column names and preprocessing
if maintenance_duration_column in df.columns:
    df[maintenance_duration_column] = pd.to_numeric(df[maintenance_duration_column], errors='coerce')
    df.dropna(subset=[maintenance_duration_column], inplace=True)
else:
    st.error(f"Column '{maintenance_duration_column}' not found in the data.")
    st.stop()

In [27]:
# Streamlit app
st.title("Machinery Maintenance Information and Prediction")

DeltaGenerator()

In [28]:
# Display the data
st.header("Maintenance Records")
st.table(df)

DeltaGenerator()

In [29]:

# One-hot encode the categorical features
encoder = OneHotEncoder(sparse_output=False)
encoded_features = encoder.fit_transform(df[[department_column, issue_column]])



In [30]:

# Create a DataFrame with the encoded features
encoded_feature_names = encoder.get_feature_names_out([department_column, issue_column])
encoded_df = pd.DataFrame(encoded_features, columns=encoded_feature_names)


In [31]:

# Combine the encoded features with the numeric feature
X = pd.concat([df[[machine_id_column]], encoded_df], axis=1)
y = df[maintenance_duration_column]


In [32]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [33]:
# Train the model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [34]:
# Train the model using statsmodels
X_train_sm = sm.add_constant(X_train)  # adding a constant
ols_model = sm.OLS(y_train, X_train_sm).fit()


In [35]:
# Make predictions using sklearn
y_pred = model.predict(X_test)

In [36]:
# Model evaluation
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

In [37]:
# Display model performance
st.header("Model Performance")
st.write("**Scikit-learn Linear Regression Model**")
st.write(f"Mean Absolute Error (MAE): {mae}")
st.write(f"Mean Squared Error (MSE): {mse}")
st.write(f"Root Mean Squared Error (RMSE): {rmse}")

st.write("**Statsmodels OLS Regression Model**")
st.write(f"Mean Absolute Error (MAE): {mae_sm}")
st.write(f"Mean Squared Error (MSE): {mse_sm}")
st.write(f"Root Mean Squared Error (RMSE): {rmse_sm}")


NameError: name 'mae_sm' is not defined

In [ ]:
# Make predictions using statsmodels
X_test_sm = sm.add_constant(X_test)
y_pred_sm = ols_model.predict(X_test_sm)


In [ ]:
# Model evaluation using statsmodels
mae_sm = mean_absolute_error(y_test, y_pred_sm)
mse_sm = mean_squared_error(y_test, y_pred_sm)
rmse_sm = np.sqrt(mse_sm)

In [ ]:
st.write("Statsmodels OLS Regression Summary")
st.text(ols_model.summary())
ols_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                  OLS Regression Results                                 
=========================================================================================
Dep. Variable:     ระยะเวลาในใช้น้ำยา /แบต (วัน)   R-squared:                       0.909
Model:                                       OLS   Adj. R-squared:                  0.905
Method:                            Least Squares   F-statistic:                     248.8
Date:                           Wed, 17 Jul 2024   Prob (F-statistic):               0.00
Time:                                   11:50:53   Log-Likelihood:                -7296.5
No. Observations:                           1841   AIC:                         1.474e+04
Df Residuals:                               1769   BIC:                         1.513e+04
Df Model:                                     71                                         
Covariance Type:                       nonrobust                                         
==============================================================================================================
                                                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------
const                                         63.1361      1.288     49.024      0.000      60.610      65.662
หมายเลขเครื่อง                                 0.0035      0.003      1.311      0.190      -0.002       0.009
แผนก_7C พักแพทย์                              16.0325      2.419      6.628      0.000      11.288      20.777
แผนก_BIH_ICU6                                 -7.2200      3.001     -2.406      0.016     -13.107      -1.333
แผนก_BIH_ICU7                                 -8.6633      3.368     -2.572      0.010     -15.269      -2.057
แผนก_BIH_ICU8                                 -7.8508      3.238     -2.425      0.015     -14.201      -1.500
แผนก_BIH_Ward4S1                              -8.3949      3.956     -2.122      0.034     -16.154      -0.636
แผนก_BIH_Ward4S2                              -8.6274      2.910     -2.965      0.003     -14.334      -2.920
แผนก_BIH_เวรเปล                               -8.0526      3.749     -2.148      0.032     -15.406      -0.699
แผนก_BRAIN CENTER (สมอง)                      -7.9756      3.769     -2.116      0.034     -15.368      -0.583
แผนก_BREAST CENTER(เต้านม)                    -9.3725      3.594     -2.608      0.009     -16.422      -2.323
แผนก_CANCER SURGERY(ศัลยกรรมมะเร็ง)           -2.5772      3.931     -0.656      0.512     -10.287       5.133
แผนก_CARDIAC REHABILITATION CENTER             8.2088      3.733      2.199      0.028       0.888      15.530
แผนก_CARDIOTHORACIC SURGERY(ศัลยกรรมหัวใจ)    10.5945      4.103      2.582      0.010       2.548      18.641
แผนก_CARDIOVASCULAR CLINIC                     6.3653      3.685      1.727      0.084      -0.863      13.593
แผนก_CATH LAB                                  6.2246      1.596      3.900      0.000       3.094       9.355
แผนก_CSSD                                      4.8059      1.506      3.190      0.001       1.851       7.760
แผนก_Chemo & IV Pharmacy Department            6.1172      3.903      1.567      0.117      -1.538      13.773
แผนก_DENTAL (ทันตกรรม)                         5.2328      1.318      3.969      0.000       2.647       7.819
แผนก_DM(เบาหวาน)                              13.1507      2.165      6.076      0.000       8.905      17.396
แผนก_EMERGENCY (ER)                           15.7562      2.701      5.834      0.000      10.459      21.053
แผนก_HDC 1R                                    9.8009      3.905      2.510      0.012       2.142      17.459
แผนก_HDC 5R                                   15.1742      2.848      5.328      0.000       9.588      20.761
แผนก_HDC 6R                                   14.1675      2.983      4.749 

In [ ]:
# Predict for a new input
st.header("Predict Time Until Maintenance Issue")
machine_id = st.number_input("Enter Machine ID:", min_value=int(df[machine_id_column].min()), max_value=int(df[machine_id_column].max()))
selected_department = st.selectbox("Select Department", df[department_column].unique())
selected_issue = st.selectbox("Select Issue", df[issue_column].unique())

In [ ]:
# Prepare the input data for prediction
input_data = pd.DataFrame([[machine_id, selected_department, selected_issue]], columns=[machine_id_column, department_column, issue_column])
input_encoded = encoder.transform(input_data[[department_column, issue_column]])
input_encoded_df = pd.DataFrame(input_encoded, columns=encoded_feature_names)
input_final = pd.concat([input_data[[machine_id_column]], input_encoded_df], axis=1)


In [ ]:
input_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   หมายเลขเครื่อง  1 non-null      int64 
 1   แผนก            1 non-null      object
 2   ปัญหา           1 non-null      object
dtypes: int64(1), object(2)
memory usage: 156.0+ bytes


In [ ]:

if st.button("Predict"):
    # Prediction using sklearn model
    prediction = model.predict(input_final)
    st.write(f"Predicted Time Until Maintenance Issue (sklearn): {prediction[0]:.2f} days")

    # Prediction using statsmodels model
    input_final_sm = np.squeeze(input_final)
    prediction_sm = ols_model.predict(input_final_sm)
    st.write(f"Predicted Time Until Maintenance Issue (statsmodels): {prediction_sm[0]:.2f} days")


In [ ]:
# Filter by machine type
st.sidebar.header("Filter by Machine Type")
selected_machine_type = st.sidebar.selectbox("Select Machine Type", df[department_column].unique())
filtered_data = df[df[department_column] == selected_machine_type]
st.header(f"Records for Machine Type: {selected_machine_type}")
st.table(filtered_data)

DeltaGenerator()

In [ ]:

# สมมุติว่าคุณมีข้อมูล input_final ที่ต้องการใช้
# input_final = ข้อมูลอินพุตของคุณในรูปแบบ numpy array หรือ pandas DataFrame

# ตรวจสอบขนาดของ input_final
print(f"Shape of input_final: {input_final.shape}")

# ตรวจสอบจำนวนตัวแปร (features) ใน input_final
n_features = input_final.shape[1]
print(f"Number of features in input_final: {n_features}")

# เพิ่มค่า bias (intercept) ใน input_final
input_final_sm = sm.add_constant(input_final)

# ตรวจสอบขนาดของ input_final_sm หลังจากเพิ่มค่า bias
print(f"Shape of input_final_sm: {input_final_sm.shape}")

# สมมุติว่า ols_model เป็นโมเดลที่ฝึกแล้ว
# ols_model = โมเดลการถดถอยเชิงเส้นที่ฝึกแล้ว

# ตรวจสอบขนาดของพารามิเตอร์ใน ols_model
print(f"Shape of ols_model.params: {ols_model.params.shape}")

# ยืนยันว่าขนาดของ input_final_sm และ ols_model.params ตรงกัน
if input_final_sm.shape[1] != ols_model.params.shape[0]:
    print("ขนาดของข้อมูลและพารามิเตอร์ไม่ตรงกัน ตรวจสอบข้อมูลและโมเดลอีกครั้ง")
else:
    # ใช้ข้อมูล input_final_sm ในการทำการพยากรณ์
    predictions = ols_model.predict(input_final_sm)
    print(predictions)

Shape of input_final: (1, 73)
Number of features in input_final: 73
Shape of input_final_sm: (1, 73)
Shape of ols_model.params: (74,)
ขนาดของข้อมูลและพารามิเตอร์ไม่ตรงกัน ตรวจสอบข้อมูลและโมเดลอีกครั้ง


In [ ]:
df

,แผนก,หมายเลขเครื่อง,ปัญหา,ระยะเวลาในใช้น้ำยา /แบต (วัน),วันที่
0,WARD 5C,2,น้ำยาหมด,23,2024-06-30
1,WARD 5C,3,น้ำยาหมด,28,2024-06-30
2,WARD 5C,3,น้ำยาหมด,30,2024-06-30
3,WARD 5C,2,น้ำยาหมด,40,2024-06-30
4,WARD 5C,3,น้ำยาหมด,25,2024-06-30
...,...,...,...,...,...
2297,KIDNEY(ศูนย์โรคไต),1473,แบตเตอร์รี่หมด,153,2024-01-08
2298,KIDNEY(ศูนย์โรคไต),1474,แบตเตอร์รี่หมด,169,2024-06-18
2299,KIDNEY(ศูนย์โรคไต),1475,แบตเตอร์รี่หมด,156,2024-05-30
2300,KIDNEY(ศูนย์โรคไต),1476,แบตเตอร์รี่หมด,157,2024-05-30


In [38]:
df2 = pd.read_excel(data_path, sheet_name="แผนกที่ติดตั้ง Dispenser")
df2

,แผนก,หมายเลขเครื่องที่ติดตั้ง
0,WARD 5C,1
1,WARD 5C,2
2,WARD 5C,3
3,WARD 5C,4
4,WARD 5C,5
...,...,...
1995,BIH_X-RAY 2S2,1996
1996,BIH_X-RAY 2S3,1997
1997,BIH_X-RAY 2S4,1998
1998,BIH_X-RAY 2S5,1999


In [ ]:
df2 

,แผนก,หมายเลขเครื่องที่ติดตั้ง
0,WARD 5C,1
1,WARD 5C,2
2,WARD 5C,3
3,WARD 5C,4
4,WARD 5C,5
...,...,...
1995,BIH_X-RAY 2S2,1996
1996,BIH_X-RAY 2S3,1997
1997,BIH_X-RAY 2S4,1998
1998,BIH_X-RAY 2S5,1999


In [39]:



# สร้าง DataFrame ที่ทำซ้ำข้อมูล และเพิ่มคอลัมน์ 'ปัญหา'
df2= df2.loc[df2.index.repeat(2)].reset_index(drop=True)
df2



,แผนก,หมายเลขเครื่องที่ติดตั้ง
0,WARD 5C,1
1,WARD 5C,1
2,WARD 5C,2
3,WARD 5C,2
4,WARD 5C,3
...,...,...
3995,BIH_X-RAY 2S4,1998
3996,BIH_X-RAY 2S5,1999
3997,BIH_X-RAY 2S5,1999
3998,BIH_X-RAY 2S6,2000


In [40]:
# Add a new column 'ปัญหา' with alternating values
df2['ปัญหา'] = ['น้ำยาหมด', 'แบตเตอร์รี่หมด'] * (len(df2) // 2)

In [41]:
df2

,แผนก,หมายเลขเครื่องที่ติดตั้ง,ปัญหา
0,WARD 5C,1,น้ำยาหมด
1,WARD 5C,1,แบตเตอร์รี่หมด
2,WARD 5C,2,น้ำยาหมด
3,WARD 5C,2,แบตเตอร์รี่หมด
4,WARD 5C,3,น้ำยาหมด
...,...,...,...
3995,BIH_X-RAY 2S4,1998,แบตเตอร์รี่หมด
3996,BIH_X-RAY 2S5,1999,น้ำยาหมด
3997,BIH_X-RAY 2S5,1999,แบตเตอร์รี่หมด
3998,BIH_X-RAY 2S6,2000,น้ำยาหมด


In [44]:
df2 =df2[['หมายเลขเครื่องที่ติดตั้ง', 'แผนก', 'ปัญหา']]
df2

,หมายเลขเครื่องที่ติดตั้ง,แผนก,ปัญหา
0,1,WARD 5C,น้ำยาหมด
1,1,WARD 5C,แบตเตอร์รี่หมด
2,2,WARD 5C,น้ำยาหมด
3,2,WARD 5C,แบตเตอร์รี่หมด
4,3,WARD 5C,น้ำยาหมด
...,...,...,...
3995,1998,BIH_X-RAY 2S4,แบตเตอร์รี่หมด
3996,1999,BIH_X-RAY 2S5,น้ำยาหมด
3997,1999,BIH_X-RAY 2S5,แบตเตอร์รี่หมด
3998,2000,BIH_X-RAY 2S6,น้ำยาหมด


In [48]:
df2 = df2.rename(columns={'หมายเลขเครื่องที่ติดตั้ง': 'machine_id_column', 'แผนก': 'department_column', 'ปัญหา': 'issue_column'})

In [49]:
df2

,machine_id_column,machine_id_column,issue_column
0,1,WARD 5C,น้ำยาหมด
1,1,WARD 5C,แบตเตอร์รี่หมด
2,2,WARD 5C,น้ำยาหมด
3,2,WARD 5C,แบตเตอร์รี่หมด
4,3,WARD 5C,น้ำยาหมด
...,...,...,...
3995,1998,BIH_X-RAY 2S4,แบตเตอร์รี่หมด
3996,1999,BIH_X-RAY 2S5,น้ำยาหมด
3997,1999,BIH_X-RAY 2S5,แบตเตอร์รี่หมด
3998,2000,BIH_X-RAY 2S6,น้ำยาหมด


In [2]:
import streamlit as st
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

# Load the data
data_path = r'สถิติ Pose Repairman.xlsx'
sheet_name = 'ข้อมูลการใช้นำยา'
df = pd.read_excel(data_path, sheet_name=sheet_name, engine='openpyxl')

# Preprocess the data (convert to numeric)
df['ระยะเวลาในใช้น้ำยา /แบต (วัน)'] = pd.to_numeric(df['ระยะเวลาในใช้น้ำยา /แบต (วัน)'], errors='coerce')

# Streamlit app
st.title("Machinery Maintenance Information and Prediction")

# Define column names
department_column = 'แผนก'
machine_id_column = 'หมายเลขเครื่อง'
issue_column = 'ปัญหา'
maintenance_duration_column = 'ระยะเวลาในใช้น้ำยา /แบต (วัน)'

# One-hot encode the categorical features
encoder = OneHotEncoder(sparse_output=False)
encoded_features = encoder.fit_transform(df[[department_column, issue_column]])

# Create a DataFrame with the encoded features
encoded_feature_names = encoder.get_feature_names_out([department_column, issue_column])
encoded_df = pd.DataFrame(encoded_features, columns=encoded_feature_names)

# Combine the encoded features with the numeric feature
X = pd.concat([df[[machine_id_column]], encoded_df], axis=1)
y = df[maintenance_duration_column]

# Binarize the target variable for classification (using a threshold)
threshold = y.median()
y_binary = (y >= threshold).astype(int)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_bin, X_test_bin, y_train_bin, y_test_bin = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# Train the models
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42)
}

model_results = {}
predictions = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r_squared = r2_score(y_test, y_pred)

    model_results[name] = {"R Squared": r_squared, "MAE": mae, "MSE": mse, "RMSE": rmse}
    predictions[name] = y_pred

# Time Series Forecasting using ARIMA
# Prepare the time series data
ts_data = df[['หมายเลขเครื่อง', 'ระยะเวลาในใช้น้ำยา /แบต (วัน)']].copy()
ts_data = ts_data.groupby('หมายเลขเครื่อง').sum()

# Train the ARIMA model
arima_model = ARIMA(ts_data, order=(5, 1, 0))
arima_model_fit = arima_model.fit()

# Make predictions
arima_forecast = arima_model_fit.forecast(steps=10)
arima_rmse = np.sqrt(mean_squared_error(ts_data.values, arima_model_fit.fittedvalues))

# Display model performance
st.header("Model Performance")
for name, metrics in model_results.items():
    st.write(f"**{name}**")
    st.write(f"R Squared: {metrics['R Squared']}")
    st.write(f"Mean Absolute Error (MAE): {metrics['MAE']}")
    st.write(f"Mean Squared Error (MSE): {metrics['MSE']}")
    st.write(f"Root Mean Squared Error (RMSE): {metrics['RMSE']}")
    st.write("---")

# Visualize the predictions vs actual values
st.header("Predictions vs Actual Values")
fig, ax = plt.subplots()
ax.plot(y_test.values, label='Actual Values', color='blue')
for name, y_pred in predictions.items():
    ax.plot(y_pred, label=f'{name} Predictions')

ax.set_xlabel('Samples')
ax.set_ylabel('Maintenance Duration (days)')
ax.legend()
st.pyplot(fig)

# Predict for a new input
st.header("Predict Time Until Maintenance Issue")
machine_id = st.number_input("Enter Machine ID:", min_value=int(df[machine_id_column].min()), max_value=int(df[machine_id_column].max()))
selected_department = st.selectbox("Select Department", df[department_column].unique())
selected_issue = st.selectbox("Select Issue", df[issue_column].unique())

# Prepare the input data for prediction
input_data = pd.DataFrame([[machine_id, selected_department, selected_issue]], columns=[machine_id_column, department_column, issue_column])
input_encoded = encoder.transform(input_data[[department_column, issue_column]])
input_encoded_df = pd.DataFrame(input_encoded, columns=encoded_feature_names)
input_final = pd.concat([input_data[[machine_id_column]], input_encoded_df], axis=1)

if st.button("Predict"):
    predictions = {name: model.predict(input_final)[0] for name, model in models.items()}
    for name, prediction in predictions.items():
        st.write(f"**{name} Prediction**: {prediction:.0f} days")

# Sidebar for filters
st.sidebar.header("Filter Records for Machine")

# Select Issue Type with 'All' option
issue_types = ['All'] + list(df[issue_column].unique())
selected_Issue_type = st.sidebar.selectbox("Select Issue Type", issue_types)

# Select Department Type with 'All' option
department_types = ['All'] + list(df[department_column].unique())
selected_department_type = st.sidebar.selectbox("Select Department Type", department_types)

# Dynamic Machine Type based on selected Department
if selected_department_type == 'All':
    machine_options = ['All'] + list(df[machine_id_column].unique())
else:
    machine_options = ['All'] + list(df[df[department_column] == selected_department_type][machine_id_column].unique())
selected_machine_type = st.sidebar.selectbox("Select Machine Type", machine_options)

# Apply filters to the DataFrame
filtered_data = df.copy()
if selected_Issue_type != 'All':
    filtered_data = filtered_data[filtered_data[issue_column] == selected_Issue_type]
if selected_department_type != 'All':
    filtered_data = filtered_data[filtered_data[department_column] == selected_department_type]
if selected_machine_type != 'All':
    filtered_data = filtered_data[filtered_data[machine_id_column] == selected_machine_type]

# Display header and filtered data
st.header(f"Records for Machine: {selected_machine_type}  Issue: {selected_Issue_type} Department: {selected_department_type}")
st.table(filtered_data.reset_index(drop=True))



2024-07-18 09:52:53.829 
  command:

    streamlit run c:\Users\LOQ\OneDrive\streamlit\.venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
c:\Users\LOQ\OneDrive\streamlit\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\LOQ\OneDrive\streamlit\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\LOQ\OneDrive\streamlit\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\LOQ\OneDrive\streamlit\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index begi

DeltaGenerator()

In [4]:
# Plot predictions from all four models vs actual values
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(y_test.values, label='Actual', color='black')
colors = ['red', 'green', 'blue', 'orange']

for idx, (name, y_pred) in enumerate(predictions.items()):
    ax.plot(y_pred, label=name, color=colors[idx])

ax.set_title('Model Predictions vs Actual Values')
ax.set_xlabel('Sample Index')
ax.set_ylabel('Maintenance Duration (days)')
ax.legend()
display(fig)
fig

<Figure size 1000x600 with 1 Axes>

<Figure size 1000x600 with 1 Axes>